In [1]:
import pandas as pd
import numpy as np

# ============================================
# 0. 데이터 로드
# ============================================
file_path = "problem_data_final.xlsx"

reward = pd.read_excel(file_path, sheet_name="Reward", parse_dates=["ts"])
trade = pd.read_excel(file_path, sheet_name="Trade", parse_dates=["ts"])
ip_df = pd.read_excel(file_path, sheet_name="IP")

# 필요하면 Funding, Spec도 같이 읽을 수 있음
# funding = pd.read_excel(file_path, sheet_name="Funding", parse_dates=["ts"])
# spec = pd.read_excel(file_path, sheet_name="Spec", parse_dates=["day"])

# ============================================
# 1. Reward 요약 지표 생성
# ============================================
reward_summary = reward.groupby("account_id").agg(
    total_reward=("reward_amount", "sum"),
    reward_count=("reward_amount", "count"),
    first_reward_ts=("ts", "min"),
    last_reward_ts=("ts", "max"),
)

# ============================================
# 2. Trade 요약 지표 생성
# ============================================
trade_summary = trade.groupby("account_id").agg(
    trade_count=("symbol", "count"),
    first_trade_ts=("ts", "min"),
    last_trade_ts=("ts", "max"),
    traded_symbols=("symbol", pd.Series.nunique),
    total_notional=("amount", "sum"),
)

# ============================================
# 3. Reward와 Trade를 account 기준으로 결합
#    (Reward가 있는 계정만 분석 대상으로 삼음)
# ============================================
summary = reward_summary.join(trade_summary, how="left")

# 거래가 전혀 없는 계정이 있을 경우를 대비한 처리
summary["trade_count"] = summary["trade_count"].fillna(0)
summary["total_notional"] = summary["total_notional"].fillna(0)

# ============================================
# 4. 계정별 IP 통계 생성
#    - n_ips: 해당 계정이 사용한 서로 다른 IP 개수
#    - max_accounts_sharing_ip: 계정이 사용한 IP 중,
#      그 IP를 공유하는 계정 수의 최댓값
# ============================================
# IP별로 몇 개의 계정이 붙어있는지 계산
ip_accounts_per_ip = ip_df.groupby("ip")["account_id"].nunique().reset_index(name="accounts_on_ip")

# 원래 IP 테이블에 붙임
ip_with_counts = ip_df.merge(ip_accounts_per_ip, on="ip", how="left")

# account_id 기준으로 요약
acc_ip_stats = ip_with_counts.groupby("account_id").agg(
    n_ips=("ip", "nunique"),
    max_accounts_sharing_ip=("accounts_on_ip", "max"),
)

# summary에 결합
summary = summary.join(acc_ip_stats, how="left")

# IP 정보가 전혀 없을 경우 기본값 설정
summary["n_ips"] = summary["n_ips"].fillna(0)
summary["max_accounts_sharing_ip"] = summary["max_accounts_sharing_ip"].fillna(1)

# ============================================
# 5. 파생 지표 계산
#    - reward_per_trade: 거래 1건당 리워드
#    - reward_per_notional: 거래 금액 대비 리워드 비율
# ============================================
summary["reward_per_trade"] = summary["total_reward"] / summary["trade_count"].replace(0, np.nan)
summary["reward_per_trade"] = summary["reward_per_trade"].fillna(0)

summary["reward_per_notional"] = np.where(
    summary["total_notional"] > 0,
    summary["total_reward"] / summary["total_notional"],
    0.0
)

# ============================================
# 6. 정규화(normalization) 작업
#    각 지표를 0~1 범위로 스케일링하여 점수 계산에 사용
# ============================================

# 6-1. total_reward 정규화
max_total_reward = summary["total_reward"].max()
summary["nr_total_reward"] = summary["total_reward"] / max_total_reward if max_total_reward > 0 else 0

# 6-2. trade_count 역정규화 (거래가 적을수록 점수 높게)
tc_min = summary["trade_count"].min()

tc_max = summary["trade_count"].max()
if tc_max > tc_min:
    summary["nr_low_trade"] = 1 - (summary["trade_count"] - tc_min) / (tc_max - tc_min)
else:
    summary["nr_low_trade"] = 0

# 6-3. reward_per_notional 정규화
max_rpn = summary["reward_per_notional"].max()
summary["nr_reward_per_notional"] = summary["reward_per_notional"] / max_rpn if max_rpn > 0 else 0

# 6-4. 동일 IP 공유 정도 정규화
#     max_accounts_sharing_ip 가 1이면 혼자 쓰는 IP, 값이 클수록 여러 계정이 공유
ip_min = 1
ip_max = summary["max_accounts_sharing_ip"].max()
if ip_max > ip_min:
    summary["nr_ip_shared"] = (summary["max_accounts_sharing_ip"] - ip_min) / (ip_max - ip_min)
else:
    summary["nr_ip_shared"] = 0

# ============================================
# 7. 보너스 노림 점수 계산
#    bonus_farming_score = w1*보상크기 + w2*거래적음 + w3*보상/거래금액비율 + w4*IP공유정도
# ============================================

w1, w2, w3, w4 = 0.3, 0.3, 0.25, 0.15

summary["bonus_farming_score"] = (
    w1 * summary["nr_total_reward"] +
    w2 * summary["nr_low_trade"] +
    w3 * summary["nr_reward_per_notional"] +
    w4 * summary["nr_ip_shared"]
)

# ============================================
# 8. 결과 정렬 및 의심 계정 상위 N개 확인
# ============================================
summary_sorted = summary.sort_values("bonus_farming_score", ascending=False)

# 상위 20개 계정만 출력
result_cols = [
    "total_reward", "reward_count",
    "trade_count", "total_notional",
    "n_ips", "max_accounts_sharing_ip",
    "reward_per_trade", "reward_per_notional",
    "nr_total_reward", "nr_low_trade",
    "nr_reward_per_notional", "nr_ip_shared",
    "bonus_farming_score",
]
summary_sorted[result_cols].head(20)

# ============================================
# 9. 보너스 이상 거래 5패턴 탐지
# ============================================

# (1) 거래 없이 리워드 수령형
reward_threshold = summary["total_reward"].quantile(0.9)
summary["pattern_1"] = (summary["trade_count"] <= 1) & (summary["total_reward"] >= reward_threshold)

# (2) 다계정 보너스 수령형
summary["pattern_2"] = (
    (summary["max_accounts_sharing_ip"] >= 3) &
    (summary["trade_count"] < 3)
)

# (4) 보너스 후 비활동형
# last_reward_ts 이후 3일 내 거래 없음
summary["pattern_4"] = (
    (summary["last_trade_ts"] - summary["last_reward_ts"]).dt.days > 3
)

# (5) 거래량 대비 리워드 과다형
reward_per_trade_95 = summary["reward_per_trade"].quantile(0.95)
reward_per_notional_avg = summary["reward_per_notional"].mean()
summary["pattern_5"] = (
    (summary["reward_per_trade"] >= reward_per_trade_95) |
    (summary["reward_per_notional"] >= reward_per_notional_avg * 5)
)

# (6) IP 그룹 집중 시점형
# Reward.ts ±5분 내 동일 IP 그룹 내 리워드 집중 여부
reward["ts_5min"] = reward["ts"].dt.floor("5min")
ip_reward_merged = reward.merge(ip_df, on="account_id", how="left")

# IP별 5분 단위 그룹핑 → 계정 수 계산
ip_time_group = (
    ip_reward_merged.groupby(["ip", "ts_5min"])["account_id"]
    .nunique()
    .reset_index(name="accounts_in_5min")
)

# 5분 내 3개 이상 계정 리워드 발생 시 집중 이벤트로 간주
ip_time_suspicious = ip_time_group[ip_time_group["accounts_in_5min"] >= 3][["ip", "ts_5min"]]
ip_reward_merged["pattern_6_flag"] = ip_reward_merged.apply(
    lambda x: ((x["ip"], x["ts_5min"]) in set([tuple(v) for v in ip_time_suspicious.values])),
    axis=1
)

# 계정별로 하나라도 해당 시점 존재하면 pattern_6 = True
pattern6_accounts = ip_reward_merged[ip_reward_merged["pattern_6_flag"]]["account_id"].unique()
summary["pattern_6"] = summary.index.isin(pattern6_accounts)

# ============================================
# 10. 패턴별 상위 계정 추출
# ============================================

pattern_cols = ["pattern_1", "pattern_2", "pattern_4", "pattern_5", "pattern_6"]

for col in pattern_cols:
    detected = summary[summary[col]].sort_values("bonus_farming_score", ascending=False)
    print(f"\n=== {col} 탐지된 계정 상위 10개 ===")
    print(
        detected.head(10)[[
            "total_reward", "trade_count", "total_notional",
            "n_ips", "max_accounts_sharing_ip",
            "reward_per_trade", "reward_per_notional",
            "bonus_farming_score"
        ]]
    )

# ============================================
# 11. 중첩 패턴 기반 종합 점수 계산
# ============================================

summary["pattern_overlap_score"] = summary[pattern_cols].sum(axis=1)
summary_sorted_overlap = summary.sort_values("pattern_overlap_score", ascending=False)

# 중첩 2개 이상인 계정 상위 20개 출력
top_overlap = summary_sorted_overlap[summary_sorted_overlap["pattern_overlap_score"] >= 2]
print("\n=== 다중 이상 패턴 중첩 계정 상위 20개 ===")
print(
    top_overlap.head(20)[[
        "total_reward", "trade_count", "total_notional",
        "reward_per_trade", "reward_per_notional",
        "pattern_overlap_score", "bonus_farming_score"
    ]]
)

# ============================================
# 12. 패턴 서브스코어 및 최종 이상 점수 계산
# ============================================

# 안전한 복사
s = summary.copy()

# === 준비: 보조 정규화 변수들 ===
# reward_per_trade 정규화 (nr_reward_per_trade)
rpt_max = s["reward_per_trade"].max()
s["nr_reward_per_trade"] = s["reward_per_trade"] / rpt_max if rpt_max > 0 else 0.0

# trade_count 정규화 (0~1, 많을수록 1)
tc_min, tc_max = s["trade_count"].min(), s["trade_count"].max()
if tc_max > tc_min:
    s["norm_trade_count"] = (s["trade_count"] - tc_min) / (tc_max - tc_min)
else:
    s["norm_trade_count"] = 0.0

# IP 그룹 이벤트 횟수 (pattern_6 관련 정량화)
# ip_reward_merged, ip_time_suspicious는 이전에 생성된 상태라면 재사용; 없으면 다시 계산
if "ip_time_suspicious" not in globals():
    reward["ts_5min"] = reward["ts"].dt.floor("5min")
    ip_reward_merged = reward.merge(ip_df, on="account_id", how="left")
    ip_time_group = (
        ip_reward_merged.groupby(["ip", "ts_5min"])["account_id"]
        .nunique()
        .reset_index(name="accounts_in_5min")
    )
    ip_time_suspicious = ip_time_group[ip_time_group["accounts_in_5min"] >= 3][["ip", "ts_5min"]]
else:
    # ip_reward_merged already exists from earlier block
    pass

# account_id 별로 pattern6 이벤트 발생 횟수
acc_pattern6_counts = (
    ip_reward_merged.merge(ip_time_suspicious, on=["ip", "ts_5min"], how="inner")
    .groupby("account_id")["ts_5min"]
    .nunique()
    .rename("pattern6_event_count")
)
s = s.join(acc_pattern6_counts, how="left")
s["pattern6_event_count"] = s["pattern6_event_count"].fillna(0)

# 정규화 (pattern6_event_count -> 0~1)
p6_max = s["pattern6_event_count"].max()
s["nr_pattern6_events"] = s["pattern6_event_count"] / p6_max if p6_max > 0 else 0.0

# === 각 패턴별 severity score 정의 ===
# (1) 거래 없이 리워드 수령형: 보상 크기 + 거래적음
#   p1 = 0.6 * nr_total_reward + 0.4 * nr_low_trade
s["p1_score"] = 0.6 * s["nr_total_reward"].fillna(0) + 0.4 * s["nr_low_trade"].fillna(0)

# (2) 다계정 보너스 수령형: IP 공유정도 + 거래적음
#   p2 = 0.7 * nr_ip_shared + 0.3 * nr_low_trade
s["p2_score"] = 0.7 * s["nr_ip_shared"].fillna(0) + 0.3 * s["nr_low_trade"].fillna(0)

# (4) 보너스 후 비활동형: (days_since_last_reward_to_trade)
# compute days gap: if last_trade_ts is NaT -> treat as large gap
def days_gap(row):
    try:
        lr = row["last_reward_ts"]
        lt = row["last_trade_ts"]
        if pd.isna(lr):
            return 0  # no reward info (shouldn't happen since summary built from reward)
        if pd.isna(lt):
            return np.inf
        return (lt - lr).days
    except Exception:
        return 0

s["days_after_last_reward_to_last_trade"] = s.apply(days_gap, axis=1)

# p4: linear scale: days<=3 ->0, days>=14 ->1, else (days-3)/(14-3)
def p4_func(d):
    if d == np.inf:
        return 1.0
    if d <= 3:
        return 0.0
    if d >= 14:
        return 1.0
    return (d - 3) / (14 - 3)

s["p4_score"] = s["days_after_last_reward_to_last_trade"].apply(p4_func)

# (5) 거래량 대비 리워드 과다형: reward_per_notional & reward_per_trade
# use normalized nr_reward_per_notional (exists) and nr_reward_per_trade
s["p5_score"] = 0.6 * s["nr_reward_per_notional"].fillna(0) + 0.4 * s["nr_reward_per_trade"].fillna(0)

# (6) IP 그룹 집중 시점형: nr_pattern6_events (정규화)
# p6 = nr_pattern6_events (already 0~1)
s["p6_score"] = s["nr_pattern6_events"].fillna(0)

# === 패턴 스코어 정리: 패턴 플래그도 고려하여 최종 패턴 스코어 반영 ===
# (패턴 플래그가 False면 해당 pX_score를 낮춰 신호 약화)
for i in [1,2,4,5,6]:
    flag_col = f"pattern_{i}"
    pcol = f"p{i}_score"
    s[pcol] = s[pcol] * s[flag_col].astype(float)  # flag False -> 0

# === 패턴 서브스코어 결합 ===
pattern_score_cols = ["p1_score","p2_score","p4_score","p5_score","p6_score"]
# 안전: 결측 -> 0
s[pattern_score_cols] = s[pattern_score_cols].fillna(0)

# 평균 기반 합성(패턴별 동등 가중) -> pattern_aggregate_score
s["pattern_aggregate_score"] = s[pattern_score_cols].mean(axis=1)

# === final anomaly score ===
# Combine original bonus_farming_score and pattern_aggregate_score.
# weighting: alpha for original, (1-alpha) for aggregate. (alpha 선택: 0.5)
alpha = 0.5
s["final_anomaly_score"] = alpha * s["bonus_farming_score"].fillna(0) + (1 - alpha) * s["pattern_aggregate_score"].fillna(0)

# === detected patterns 문자열 컬럼 추가 ===
def collect_patterns(row):
    pats = []
    if row.get("pattern_1", False):
        pats.append("거래없이_리워드수령형")
    if row.get("pattern_2", False):
        pats.append("다계정_보너스수령형")
    if row.get("pattern_4", False):
        pats.append("보너스후_비활동형")
    if row.get("pattern_5", False):
        pats.append("거래량대비_리워드과다형")
    if row.get("pattern_6", False):
        pats.append("IP그룹_집중시점형")
    return ",".join(pats) if pats else "None"

s["detected_patterns"] = s.apply(collect_patterns, axis=1)

# 정렬 및 상위 20 추출
top20_final = s.sort_values("final_anomaly_score", ascending=False).head(20)

# === 출력: top20 with pattern list and scores ===
out_cols = [
    "total_reward","reward_count","trade_count","total_notional",
    "reward_per_trade","reward_per_notional",
    "p1_score","p2_score","p4_score","p5_score","p6_score",
    "pattern_aggregate_score","bonus_farming_score","final_anomaly_score","detected_patterns"
]
print("=== Final Top 20 (final_anomaly_score 기준) ===")
print(top20_final[out_cols])

# 또한, 패턴별 상위 리스트(예: 각 패턴의 Top10 by final score) 출력
for i in [1,2,4,5,6]:
    pc = f"pattern_{i}"
    detected = s[s[pc]].sort_values("final_anomaly_score", ascending=False)
    print(f"\n=== {pc} Top 10 (by final_anomaly_score) ===")
    print(detected.head(10)[out_cols])



=== pattern_1 탐지된 계정 상위 10개 ===
Empty DataFrame
Columns: [total_reward, trade_count, total_notional, n_ips, max_accounts_sharing_ip, reward_per_trade, reward_per_notional, bonus_farming_score]
Index: []

=== pattern_2 탐지된 계정 상위 10개 ===
Empty DataFrame
Columns: [total_reward, trade_count, total_notional, n_ips, max_accounts_sharing_ip, reward_per_trade, reward_per_notional, bonus_farming_score]
Index: []

=== pattern_4 탐지된 계정 상위 10개 ===
              total_reward  trade_count  total_notional  n_ips  \
account_id                                                       
A_f96ede8d34    796.028294          802    1.360494e+07     16   
A_ebdb869fd3    159.991358         1169    1.309901e+05    177   
A_d444580218    336.895867          440    1.003083e+07      4   
A_48fbd03d61    106.000000          170    6.671425e+06     26   
A_cab220ef3c   1773.536296         2183    3.491002e+08     26   
A_8860be39b3     10.000000          239    6.338027e+07     19   
A_26ffea8fd9    243.544820     